In [1]:
%%bash
pip install -q openvino

In [2]:
import os
import cv2
import random as r
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from typing import Union
from openvino.runtime import Core

In [3]:
def preprocess(image: np.ndarray, width: int, height: int, model_name: str="facenet") -> np.ndarray:
    if model_name == "facenet":
        image = cv2.resize(src=image, dsize=(width, height), interpolation=cv2.INTER_AREA)
    else:
        image = cv2.resize(src=image, dsize=(width, height), interpolation=cv2.INTER_AREA).transpose(2, 0, 1)
    return np.expand_dims(image, axis=0)


def setup(target: str, model_path: str) -> tuple:
    ie = Core()
    model = ie.read_model(model=model_path)
    model = ie.compile_model(model=model, device_name=target)

    input_layer = next(iter(model.inputs))
    output_layer = next(iter(model.outputs))

    return model, input_layer, output_layer, \
           (input_layer.shape[0], input_layer.shape[1], input_layer.shape[2], input_layer.shape[3])


def detect_faces(
    model, 
    output_layer, 
    image: np.ndarray, 
    w: int, 
    h: int,
    threshold: float=0.9,
) -> tuple:
    result = model(inputs=[image])[output_layer].squeeze()

    label_indexes: list = []
    probs: list = []
    boxes: list = []

    if result[0][0] == -1:
        return 0, None, None    
    else:
        for i in range(result.shape[0]):
            if result[i][0] == -1:
                break
            elif result[i][2] > threshold:
                label_indexes.append(int(result[i][1]))
                probs.append(result[i][2])
                boxes.append([int(result[i][3] * w), \
                              int(result[i][4] * h), \
                              int(result[i][5] * w), \
                              int(result[i][6] * h)])
            else:
                pass
        
    return label_indexes, probs, boxes

In [4]:
BASE_PATH: str = "../input/11-785-f22-hw2p2-verification/verification"
clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(5, 5))

known = sorted(os.listdir(f"{BASE_PATH}/known"))
unknown_test = sorted(os.listdir(f"{BASE_PATH}//unknown_dev"))
df = pd.read_csv(f"{BASE_PATH}/dev_identities.csv")

CS_1: list = []
CS_2: list = []    
AVG_: list = []    

d_model, _, d_output_layer, (_, _, d_H, d_W) = setup("CPU", "../input/ovfacemodels/detect_model.xml")
r_model, _, r_output_layer, (_, r_H, r_W, _) = setup("CPU", "../input/ovfacemodels/recog_model_2.xml")

### **Known**

In [5]:
if not os.path.exists("known-embeddings"): os.makedirs("known-embeddings")

for identity in known:
    filename = os.listdir(f"{BASE_PATH}/known/{identity}")[0]
    
    image = cv2.imread(f"{BASE_PATH}/known/{identity}/{filename}", cv2.IMREAD_COLOR)
    for i in range(3): image[:, :, i] = clahe.apply(image[:, :, i])
    image = preprocess(image, r_W, r_H, "facenet")
    embeddings = r_model(inputs=[image])[r_output_layer]
    np.save(f"{identity}_fr.npy", embeddings)
    
    del image, embeddings
    
    image = cv2.imread(f"{BASE_PATH}/known/{identity}/{filename}", cv2.IMREAD_COLOR)
    h, w, _ = image.shape 
    for i in range(3): image[:, :, i] = clahe.apply(image[:, :, i])
    temp_image = image.copy()
    image = preprocess(image, d_W, d_H, "other")
    _, _, boxes = detect_faces(d_model, d_output_layer, image, w, h)
    if len(boxes) != 0:
        for i in range(len(boxes)): 
            if boxes[0][i] < 0: 
                boxes[0][i] = 0
        image = preprocess(temp_image[boxes[0][1]:boxes[0][3], boxes[0][0]:boxes[0][2], :], r_W, r_H, "facenet")
    else:
        image = preprocess(temp_image, r_W, r_H, "facenet")
    embeddings = r_model(inputs=[image])[r_output_layer]
    np.save(f"known-embeddings/{identity}_fd_fr.npy", embeddings)
    
    del image, temp_image, embeddings

### **Unknown Test**

In [6]:
if not os.path.exists("unknown-test-embeddings"): os.makedirs("unknown-test-embeddings")

filenames = os.listdir(f"{BASE_PATH}/unknown_test")

for filename in filenames:
    
    image = cv2.imread(f"{BASE_PATH}/unknown_test/{filename}", cv2.IMREAD_COLOR)
    for i in range(3): image[:, :, i] = clahe.apply(image[:, :, i])
    image = preprocess(image, r_W, r_H, "facenet")
    embeddings = r_model(inputs=[image])[r_output_layer]
    np.save(f"{filename}_fr.npy", embeddings)
    
    del image, embeddings
    
    image = cv2.imread(f"{BASE_PATH}/unknown_test/{filename}", cv2.IMREAD_COLOR)
    h, w, _ = image.shape 
    for i in range(3): image[:, :, i] = clahe.apply(image[:, :, i])
    temp_image = image.copy()
    image = preprocess(image, d_W, d_H, "other")
    _, _, boxes = detect_faces(d_model, d_output_layer, image, w, h)
    if len(boxes) != 0:
        for i in range(len(boxes)): 
            if boxes[0][i] < 0: 
                boxes[0][i] = 0
        image = preprocess(temp_image[boxes[0][1]:boxes[0][3], boxes[0][0]:boxes[0][2], :], r_W, r_H, "facenet")
    else:
        image = preprocess(temp_image, r_W, r_H, "facenet")
    embeddings = r_model(inputs=[image])[r_output_layer]
    np.save(f"unknown-test-embeddings/{filename}_fd_fr.npy", embeddings)
    
    del image, temp_image, embeddings